In [150]:
import numpy as np
from scipy.optimize import minimize_scalar
import random
import scipy.optimize

In [151]:
def rosenbrock(x):
    
    global calls
    calls += 1
    
    n = len(x)
    res = 0
    for i in range(n - 1):
        res += 100.0 * (x[i + 1] - x[i]**2)**2 + (1 - x[i])**2
    return res

In [152]:
def random_between(l, r):
    return l + random.random() * (r - l)

In [153]:

calls = 0


epsilon = 1e-6
delta = 1e-5
n_sum = int (input("enter N:"))

#x0 = np.array([float(val) for val in input("Введіть початкову точку x0 через пробіл: ").split()])
#x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0])
x0 = np.full(n_sum, 0)
max_iterations = int(input("Введіть максимальну кількість ітерацій (K < 1000): "))

In [154]:
def rosenbrock_gradient(x):
    
    global calls
    calls += len(x)
    
    if len(x) < 2:
        raise ValueError("Для обчислення градієнта x повинен мати розмірність принаймні 2.")
    
    n = len(x)
    gradient = np.zeros_like(x)
    gradient[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    gradient[1:-1] = 200 * (x[1:-1] - x[:-2]**2) - 400 * x[1:-1] * (x[2:] - x[1:-1]**2) - 2 * (1 - x[1:-1])
    gradient[-1] = 200 * (x[-1] - x[-2]**2)
    return gradient

In [155]:
# def rosenbrock_hessian(x):
    
#     n = len(x)
#     hessian = np.zeros((n, n))
#     hessian[0, 0] = 1200 * x[0]**2 - 400 * x[1] + 2
#     hessian[0, 1] = -400 * x[0]
#     hessian[1, 0] = -400 * x[0]
    
#     for i in range(1, n - 1):
#         hessian[i, i] = 200
#         hessian[i, i + 1] = -400 * x[i]
#         hessian[i + 1, i] = -400 * x[i]
    
#     hessian[-1, -1] = 200
#     return hessian

def rosen_hess(x):
    
    global calls
    calls += n_sum * (n_sum + 1) / 2
    
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H


In [156]:
def golden_section_search(f, a, b, tol=1e-5):
    gr = (np.sqrt(5) + 1) / 2 - 1
    c = b - gr * (b - a)
    d = a + gr * (b - a)

    while abs(c - d) > tol:
        if f(c) < f(d):
            b = d
        else:
            a = c

        c = b - gr * (b - a)
        d = a + gr * (b - a)

    return (b + a) / 2

In [157]:
def newton_method(n, epsilon, delta, K):
    
    #x0 = np.random.uniform(0, 0.9, n)
    x = x0
    k = 0

    while k < K:
        gradient = rosenbrock_gradient(x)
        hessian = rosen_hess(x)
        p = -np.linalg.solve(hessian, gradient)

        if np.linalg.norm(p) < epsilon:
            break

        objective_function = lambda alpha: rosenbrock(x + alpha * p)
        alpha_k = golden_section_search(objective_function, 0, 1, tol=delta)

        x = x + alpha_k * p
        k += 1

    return x, rosenbrock(x), np.linalg.norm(p), k

In [158]:
result = newton_method(n_sum, epsilon, delta, max_iterations)

print("Останнє значення x_{k+1}: ", result[0])
print("Значення f(x_{k+1}): ", result[1])
print("Норма вектора p_k: ", result[2])
print("Кількість ітерацій: ", result[3])
print("Кількість звернень до градієнта та гессіана: ", calls)

Останнє значення x_{k+1}:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Значення f(x_{k+1}):  4.5297169353072806e-21
Норма вектора p_k:  1.1281722752795579e-10
Кількість ітерацій:  157
Кількість звернень до градієнта та гессіана:  820295.0
